# This notebook is for testing purposes , It is not the final deliverable.
## Contents:
0. [Message Encoding with message blocks](#0)
1. [Diffie-Hellman Key Exchange](#1)
2. [AlGamal](#2)
3. [Verifying Signature](#3)
4. [Handling Messages with AES](#4)
5. [Integration to chat.py](#5)

0. [Message Encoding with message blocks](#0)

### Import relevant libraries

In [1]:
import base64

from helpers.dh_gamal import deffie_hellman
from helpers.tools import from_file
import numpy as np
import hashlib

%reload_ext autoreload
%autoreload 2

### Message Encoding with message blocks

The main reason to use Base64 is to be trasmit data over a channel that doesn't allow binary data transfer. That said, now it should be clear that you can have a stream encoded in Base64 that rapresent a stream UTF8 encoded.

[What is the difference between utf-8 and base64](https://stackoverflow.com/questions/3866316/whats-the-difference-between-utf8-utf16-and-base64-in-terms-of-encoding#:~:text=But%20in%20languages%20where%20there's,to%20keep%20the%20concerns%20separate.&text=%2B1%20Base64%20is%20a%20way,ways%20to%20encode%20Unicode%20text.)

#### Trying different encodings and decodings (Done)

In [ ]:
message = "hello world hello baba hello mama hello teta hello geddo"

In [ ]:
print("Original Message: " , message)

print("#############################################################")
print("UTF-ENCDOING")
print("#############################################################")
print("Message with UTF-8 encoding: ",message.encode('utf-8'))
print("Message with UTF-16 encoding: ",message.encode('utf-16'))
print("Message with UTF-32 encoding: ",message.encode('utf-32'))


print("#############################################################")
print("INT.FROM_BYTES little endian encoding")
print("#############################################################")
# testing int.from_bytes
print(int.from_bytes(message.encode('utf-8'), 'little'))
print(int.from_bytes(message.encode('utf-16'), 'little'))
print(int.from_bytes(message.encode('utf-32'), 'little'))


print("#############################################################")
print("Base64 Encoding")
print("#############################################################")
# encode with base64
message_bytes = message.encode('utf-8')
print("Message with base64 encoding: ", base64.b64encode(message_bytes))
print("Message with a85 encoding: ", base64.a85encode(message_bytes))
print("Message with b85 encoding: ", base64.b85encode(message_bytes))
print("Message with b16 encoding: ", base64.b16encode(message_bytes))
print("Message with b32 encoding: ", base64.b32encode(message_bytes))
print("Message with b85 encoding: ", base64.b85encode(message_bytes))
print("Message with b32hex encoding: ", base64.b32hexencode(message_bytes))
print("Message with standard base64 encoding: ", base64.standard_b64encode(message_bytes))
print("Message with urlsafe base64 encoding: ", base64.urlsafe_b64encode(message_bytes))

print("#############################################################")
print("INT FROM_BYTES little endian encoding")
print("#############################################################")
# testing int.from_bytes
print(int.from_bytes(base64.b64encode(message_bytes), 'little'))
print(int.from_bytes(base64.a85encode(message_bytes), 'little'))
print(int.from_bytes(base64.b85encode(message_bytes), 'little'))
print(int.from_bytes(base64.b16encode(message_bytes), 'little'))
print(int.from_bytes(base64.b32encode(message_bytes), 'little'))
print(int.from_bytes(base64.b85encode(message_bytes), 'little'))
print(int.from_bytes(base64.b32hexencode(message_bytes), 'little'))
print(int.from_bytes(base64.standard_b64encode(message_bytes), 'little'))
print(int.from_bytes(base64.urlsafe_b64encode(message_bytes), 'little'))

print(message.encode('utf-8').decode('utf-8'))
print(base64.b85decode(base64.b85encode(message.encode('utf-8'))).decode('utf-8'))

### 1. [Diffie-Hellman Key Exchange](#1) (Done)

In [2]:
q_dh , a_dh , q_gamal , a_gamal = from_file("agree.txt")

In [3]:
np.random.seed(42)

X_a = np.random.randint(2, q_dh)
print("Alice Private Key for DH: ", X_a)
X_b = np.random.randint(2, q_dh)
print("Bob Private Key for DH: ", X_b)
Y_a = deffie_hellman(prime=q_dh, generator=a_dh, private_key=X_a)
print("Alice Public Key for DH: ", Y_a)
Y_b = deffie_hellman(prime=q_dh, generator=a_dh, private_key=X_b)
print("Bob Public Key for DH: ", Y_b)



Alice Private Key for DH:  104
Bob Private Key for DH:  437
Alice Public Key for DH:  385
Bob Public Key for DH:  674


In [4]:
X_a2 = np.random.randint(2,q_gamal)
print("Alice Private Key for Al-Gamal: ", X_a2)
X_b2 = np.random.randint(2, q_gamal)
print("Bob Private Key for Al-Gamal: ", X_b2)
Y_a2 = deffie_hellman(prime=q_gamal, generator=a_gamal, private_key=X_a2)
print("Alice Public Key for Al-Gamal: ", Y_a2)
Y_b2 = deffie_hellman(prime=q_gamal, generator=a_gamal, private_key=X_b2)
print("Bob Public Key for Al-Gamal: ", Y_b2)

Alice Private Key for Al-Gamal:  16
Bob Private Key for Al-Gamal:  12
Alice Public Key for Al-Gamal:  5
Bob Public Key for Al-Gamal:  11


In [8]:
def send_signature(public,generator,prime):
    M = public
    k = np.random.randint(2,prime)
    K = pow(public,k,prime)
    print("K: ", K)
    C1 = pow(generator,k,prime)
    C2 = (M*K) % prime
    return C1,C2

def verify_signature(public,private,prime,C1,C2):
    K = pow(C1,private,prime)
    print("K: ", K)
    for j in range(1,prime):
        if (K*j) % prime == 1:
            break
    print("M recovered " , C2*j % prime)
    return ((C2*j) % prime) == public
    

In [9]:
c1 , c2 = send_signature(Y_a,a_gamal,q_gamal)
print("Signature sent by Bob: ", Y_a,c1,c2)
print("Signature verified by Alice: ", verify_signature(Y_a,X_b,q_gamal,c1,c2))

K:  184
Signature sent by Alice:  385 640 67
K:  41
M recovered  113
Signature verified by Bob:  False


In [ ]:
# let M = Y_a 
M = Y_a
print(M)
# m = SHA1(M) [pick N lsb bits for 1< m <q_dh]
m = int.from_bytes(hashlib.sha1(M.to_bytes(16, 'big')).digest(), 'big') % q_dh
# choose some random K 
k = np.random.randint(2, q_dh)
print("k " , k)
# K = Y_a^k mod q
K = pow(Y_a, k, q_dh)
print("K: ", K)
# C1 = a^K mod q
C1 = a_dh**k % q_dh
# C2 = K * m mod q
C2 = (K*M) % (q_dh)
print("Signature: ", C1, C2)


In [ ]:
# recovering Message 
K = pow(C1, X_a, q_dh)
print(K)

# time to find K-1
for j in range(2,q_dh):
    if (K*j) % q_dh == 1:
        print("Found K-1: ", j)
        break

M_recovered = (C2 * j) % q_dh

print(M_recovered)
print(M_recovered == M)

### 2. [AlGamal](#2) (Done)

##### testing rsa on wide message

##### trying sha1

### 3. [Verifying signature](#3)

### 4. [Handling Messages](#4)

### 5. [Integration to chat.py](#5)